Raw data are 42.6 GB in Edf format. Convert to csv for smaller size, and easier processing.

1. Import mne and pandas library.
2. mne.io.read_raw_edf to read each Edf file.
3. mne.io.read_raw_edf.get_data to retrive read data.
4. convert to numpy using  numpy.vstack. Vertical stack converts dimensions into shape of samples x features.
5. save as csv using pandas.DataFrame.to_csv

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import mne # reads edf format
import glob
import random
import re
import wfdb # waveform database library wfdb.rdann reads annotation of physiology annotated data in ECG, EEG etc


In [2]:
# Each folder is assigned an integer from 0 to 24. There are 23 patients. 22 have a single folder, and 1 has two folders.
# Retrieve assigned folder number from file

path = r'data\raw_data\chb-mit-scalp-eeg-database-1.0.0'
data_folders = sorted(glob.glob(os.path.join(path, '*[0-9]')))

def file_id(folder):
    return [ name[-2:] for name in [file.rsplit('\\',2)[-1] for file in folder]]
print("Case ID: ")
print(file_id(data_folders))

Case ID: 
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [5]:
# seizure  marking is in name of edf file, not a separate file
# split files for training and testing
train_test_ratio = 0.8
random.seed(80)
train_folders = sorted(random.sample(data_folders, round(len(data_folders) * train_test_ratio)))
test_folders = sorted([ file for file in data_folders if file not in train_folders])
print(f"train_folders' IDs: {file_id(train_folders)}, contains {len(train_folders)} files")
print(f"test_folders' IDs: {file_id(test_folders)}, contains {len(test_folders)} files")


train_folders' IDs: ['01', '02', '04', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24'], contains 19 files
test_folders' IDs: ['03', '05', '06', '11', '21'], contains 5 files


In [6]:
# Retrieve edf files

train_files = [ file for folder in train_folders for file in glob.glob(folder+'/*.edf')]
test_files = [ file for folder in test_folders for file in glob.glob(folder+'/*.edf')]

print(f"Train_files contains {len(train_files)} files")
print(f"Test_files contains {len(test_files)} files")
print(f"Random example from train_files: {random.choice(train_files)}")


Train_files contains 523 files
Test_files contains 163 files
Random example from train_files: data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_16.edf


In raw files, some files are duplicated. Some files have duplicated name.

In [5]:
# MEG and EEG library (mne) reads neurophysiological data
# dir(sample_edf)  # shows all attributes and methods

sample_edf = mne.io.read_raw_edf(train_files[0], preload=False)
sample_edf.info

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_5836\3700619184.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  sample_edf = mne.io.read_raw_edf(train_files[0], preload=False)


<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 11:42:54 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: 1 item (dict)
>

In [6]:
# identify montage used for surface electrode ( naming convention to identify each surface scalp electrode)
sample_edf.describe()

<RawEDF | chb01_01.edf, 23 x 921600 (3600.0 s), ~27 kB, data not loaded>
ch  name      type  unit        min         Q1     median         Q3        max
 0  FP1-F7    EEG   µV      -807.03     -24.03      -1.37      19.73    1037.95
 1  F7-T7     EEG   µV      -726.54     -17.00       0.20      17.00     549.55
 2  T7-P7     EEG   µV      -544.47     -13.48       0.20      14.26     588.62
 3  P7-O1     EEG   µV      -449.52     -10.74       0.59      11.53     206.11
 4  FP1-F3    EEG   µV      -882.44     -26.37      -0.98      22.86     792.19
 5  F3-C3     EEG   µV      -431.94     -14.65       0.20      15.04     402.25
 6  C3-P3     EEG   µV      -417.09     -10.35       0.20      10.74     536.65
 7  P3-O1     EEG   µV      -201.03     -13.48       0.20      14.26     229.94
 8  FP2-F4    EEG   µV      -563.61     -24.03      -0.98      20.51     700.76
 9  F4-C4     EEG   µV      -262.76     -13.48       0.20      13.48     235.41
10  C4-P4     EEG   µV      -284.64     -10.35 

In [9]:
channel_labels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',
                  'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                  'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']

#### The international 10-20 systems defines placement of scalp electrodes.
Older system use 18 pairs or 18 channels
Newer ones may have 23,24 or 26 pairs.

For this project we use 23 channels.




EEG files are a montage of amplitudes from channels.

Each channel represents scalp electrode detection of brain signal amplitude. The montage is arranged from left to right, and front of scalp to back.
The channels are labelled as

 ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',

 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',

 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']

#### Signal Extraction

Codes extensively modified from work by [Mashahiro Goton](https://www.kaggle.com/code/hemangjindal/chb-mit-eeg-dataset-my-notebook#CHB-MIT-eeg-dataset-seizure-detection-demo). \[Online] [Accesssed on 20 October 2024].

EDfToNpy class contains:
1. read_efd - read edf formatted seizure file. Identify parameters for creating numpy arrays. Ensure correct channels and calculate samples of seizure and non seizure files.
2. extract_edf - create appropriate numpy arrays using arguments obtained from read_edf method. Copy data and labels to numpy array, then save to file as npy format.
3. show_EEG - display EEG

In [3]:
class EdfToNpy:
    # Constants
    WINDOW_TIME = 10  # segment size in second
    STEP_TIME = 5     # Step size in second
    SEIZURE_PROPORTION = 0.01    # proportion of seizure
    TO_MICROVOLTS = 1e6
    channel_labels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
                      'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                      'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']

    def __init__(self, folder, save_to):
        self.save_to = save_to
        self.folder = folder

    # Read edf formatted file
    def read_edf(self):
        count = 0  # initialize the count variable
        window_size = 0  # initialize the window_size variable

        for file in self.folder:
            edf_data = mne.io.read_raw_edf(file, preload=False)
            edf_labels = edf_data.ch_names

            if sum([any([0 if re.match(c, l) is None else 1 for l in edf_labels]) for c in EdfToNpy.channel_labels]) == len(EdfToNpy.channel_labels):
                sampling_freq = int(1 / (edf_data.times[1] - edf_data.times[0]))
                window_size = sampling_freq * EdfToNpy.WINDOW_TIME
                window_stride = sampling_freq * EdfToNpy.STEP_TIME

                has_seizure = np.zeros((edf_data.n_times,))
                if os.path.exists(file + '.seizures'):
                    has_annotation = wfdb.rdann(file, 'seizures')
                    for idx in range(int(has_annotation.sample.size / 2)):
                        has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1

                has_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])

                noseizure_n_size = round(EdfToNpy.SEIZURE_PROPORTION * np.where(has_seizure_idx == 0)[0].size)
                seizure_n_size = np.where(has_seizure_idx > 0)[0].size
                count = count + noseizure_n_size + seizure_n_size  # increment count to tally total samples

            edf_data.close()

        return count, len(EdfToNpy.channel_labels), window_size

    # Segment data into 10-second window and save data to numpy data file. Also save seizure annotation to numpy label file.
    def extract_edf(self, n_samples, n_channel_labels, window_size):
        signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
        labels_np = np.zeros(n_samples, dtype=np.int32)
        count = 0  # initialize the count variable

        for number, file in enumerate(self.folder):
            edf_data = mne.io.read_raw_edf(file, preload=False)

            n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_data.ch_names]) for ch in EdfToNpy.channel_labels])
            if n_label_match == len(EdfToNpy.channel_labels):
                dict_ch_name = {sorted([ch_name for ch_name in edf_data.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in EdfToNpy.channel_labels}
                edf_data.rename_channels(dict_ch_name)

                has_seizure = np.zeros((edf_data.n_times,))
                signals_ = edf_data.get_data(picks=EdfToNpy.channel_labels) * EdfToNpy.TO_MICROVOLTS

                if os.path.exists(file + '.seizures'):
                    has_annotation = wfdb.rdann(file, 'seizures')
                    for idx in range(int(has_annotation.sample.size / 2)):
                        has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1

                sampling_freq = int(1 / (edf_data.times[1] - edf_data.times[0]))
                window_size = sampling_freq * EdfToNpy.WINDOW_TIME
                window_stride = sampling_freq * EdfToNpy.STEP_TIME
                has_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])

                noseizure_n_size = round(EdfToNpy.SEIZURE_PROPORTION * np.where(has_seizure_idx == 0)[0].size)
                seizure_n_size = np.where(has_seizure_idx > 0)[0].size

                # Non-seizure data
                temp_negative = random.sample(list(np.where(has_seizure_idx == 0)[0]), noseizure_n_size)  # sample non-seizure data correctly
                for value in temp_negative:
                    start_index = value * window_stride
                    stop_index = value * window_stride + window_size
                    signals_np[count, :, :] = signals_[:, start_index:stop_index]
                    labels_np[count] = 0
                    count = count + 1

                # Seizure data
                temp_positive = list(np.where(has_seizure_idx > 0)[0])  # sample seizure data correctly
                for value in temp_positive:
                    start_index = value * window_stride
                    stop_index = value * window_stride + window_size
                    signals_np[count, :, :] = signals_[:, start_index:stop_index]
                    labels_np[count] = 1
                    count = count + 1
            else:
                print(f"Unable to read {file}")

            edf_data.close()

        np.save(self.save_to + '_signals', signals_np)
        np.save(self.save_to + '_labels', labels_np)

    def show_eeg(self, signals):
        vertical_width = 250
        fs = 256

        fig, ax = plt.subplots()
        for i in range(signals.shape[0]):
            ax.plot(np.arange(signals.shape[-1]) / fs, signals[i, :] + i * vertical_width, linewidth=0.5, color='tab:blue')
            ax.annotate(EdfToNpy.channel_labels[i], xy=(0, i * vertical_width))
        ax.invert_yaxis()
        plt.show()

In [7]:
# 5-6 minutes to run cell
# extract data into training numpy file

# Instantiate class
train_npy =  EdfToNpy(train_files, 'train_10sec')

# Get training samples
sample_length, channel_length, window_length = train_npy.read_edf()
train_npy.extract_edf(sample_length, channel_length, window_length)

NameError: name 'train_files' is not defined

In [ ]:
# 1-2 minutes to run cell
# extract remaining portion into testing numpy file

# Instantiate class
test_npy = EdfToNpy(test_files, 'test_10sec')
# Get training samples
sample_length, channel_length, window_length = test_npy.read_edf()
test_npy.extract_edf(sample_length, channel_length, window_length)



#### First Iteration

Successfully implemented reading and converting raw edf format to numpy format during prototyping.

EdfToNumpy class is created to contain those methods.

Raw data is segmented into 10 second windows for each sample.

In [16]:
# Check numpy files are readable
# Training data
path_signal = 'train_10sec_signals.npy'
path_label = 'train_10sec_labels.npy'
if os.path.exists(path_label):
    train_labels = np.load(path_label)
    unique, count = np.unique(train_labels, return_counts=True)
    print(f"Train labels {unique} contain {count}")

# Testing  data
path_signal = 'test_10sec_signals.npy'
path_label = 'test_10sec_labels.npy'
if os.path.exists(path_label):
    test_labels = np.load(path_label)
    unique, count = np.unique(test_labels, return_counts=True)
    print(f"Test labels {unique} contains {count}")



Train labels [0 1] contain [9190  119]
Test labels [0 1] contains [2307  186]


#### Problems

Current train-test samples are splitted by folder or case number.

Each case may have different seizure type. Different seizure types have different signal signatures.

Therefore training and testing are likely to have different seizure signature.

Training sample presented to classifier will not be representative of all seizure types in the entire CHB-Mit seizure dataset.

> To overcome this initial mistake, the entire CHB-MIT dataset is segmented into 10-second window samples and converted to a single numpy signal file, and corresponding numpy label file.

In [4]:
# Check numpy files are readable
# Retrieve all edf files contained within parent folder - "data_folders"

all_edf = [ file for folder in data_folders for file in glob.glob(folder + '/*.edf')]
print(f"Total number of files is {len(all_edf)}")

Total number of files is 686


In [5]:
# 8-9 minutes to run cell

# Instantiate class
data_npy = EdfToNpy(all_edf, '10sec')
# Export all edf file data into single numpy 10-second window per sample file - "10sec_signals.npy"
# Export annotations into corresponding "10sec_labels.npy", with 1 for seizure and 0 for non-seizure. Corresponds to each signal sample.
sample_length, channel_length, window_length = data_npy.read_edf()
data_npy.extract_edf(sample_length, channel_length, window_length)

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF param

C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ip

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_17.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_19.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_20.edf...
EDF file detected
Se

C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)
C:\Users\mspla\AppData\Local\Temp\ip

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_36.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_37.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_38.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_39.edf...
EDF file detected
Se

...

C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:52: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb24\chb24_21.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:52: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb24\chb24_22.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_2308\4220419912.py:52: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=False)


####  10sec_labels.npy

In [6]:
# Check numpy files are readable

path_signal = '10sec_signals.npy'
path_label = '10sec_labels.npy'
if os.path.exists(path_signal) and os.path.exists(path_label):
    full_data = np.load(path_signal)
    full_labels = np.load(path_label)

    unique, count = np.unique(full_labels, return_counts=True)
    print(f"Labels contain {count[0]} non-seizure samples and {count[1]} seizure samples")
    print(f"Data shape is {full_data.shape}")
    print(f"Labels' shape is {full_labels.shape}")



Labels contain 6744 non-seizure samples and 2606 seizure samples
Data shape is (9350, 23, 2560)
Labels' shape is (9350,)


#### END